### -------------Yanolja crawling---------------

In [9]:
import scrapy, requests
from scrapy.http import TextResponse
import pandas as pd
import json

### 1. scrapy로 룸정보 제외한 기본정보 가져오기

In [142]:
# 1. 프로젝트 생성

In [14]:
!rm -rf yanolja_hotel
!scrapy startproject yanolja_hotel

New Scrapy project 'yanolja_hotel', using template directory '/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/templates/project', created in:
    /home/ubuntu/python3/notebook/project/crawling/crawling-repo-2/in_progress/yanolja_hotel

You can start your first spider with:
    cd yanolja_hotel
    scrapy genspider example example.com


In [15]:
!tree yanolja_hotel

yanolja_hotel
├── scrapy.cfg
└── yanolja_hotel
    ├── __init__.py
    ├── items.py
    ├── middlewares.py
    ├── pipelines.py
    ├── settings.py
    └── spiders
        └── __init__.py

2 directories, 7 files


In [16]:
# 2. xpath 찾기 : headers 설정

In [73]:
import requests #호텔 key값 얻는 모듈 생성 -> yanolja.py
import datetime

def get_hotel_keys():    
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36", 
           "X-Requested-With": "XMLHttpRequest"
              }
    today = datetime.date.today()
    checkin_date = today.strftime("%Y-%m-%d")
    checkout_date = (today + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
    region = "서울"
    num=0
    page = 1
    key_info = []
    while True:
        parent_url = f"https://www.yanolja.com/api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0\
    &checkinDate={checkin_date}&checkoutDate={checkout_date}&hotel=1&keyword=서울&lat=37.50681&lng=127.06624&page={page}&limit=20&\
    searchKeyword={region}&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes="
        req = requests.get(parent_url, headers=headers)
        response = req.json()
        #print('page=', str(apage))
        page += 1

        if response["motels"]["counts"] > 0:
            key = response["motels"]
            key_info.append(key)
        else:
            break
    hotel_keys = []
    for i in key_info:
        for j in i['lists']:
            hotel_keys.append(j['key'])        
    return hotel_keys

In [74]:
hotel_keys = get_hotel_keys()
len(hotel_keys)

416

### ------------xpath 확인 -> df확인 --------------

In [75]:
links = []
url = "https://www.yanolja.com/hotel/"
for hotel_key in hotel_keys:
    hotel_url = url + hotel_key
    links.append(hotel_url)
len(links)

416

In [76]:
link = links[0]
headers = {"user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
req = requests.get(link, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8")
response

<200 https://www.yanolja.com/hotel/3013410>

In [77]:
import json

if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
else:    
    room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
room_type

['Premier Twin',
 'Premier King',
 'Superior Single',
 'Premier Twin',
 'Premier Twin',
 'Premier King',
 'Premier King',
 'Premier Twin',
 'Premier King',
 'Premier Twin',
 'Premier King',
 'Premier Twin',
 'Premier King',
 'Premier Twin',
 'Premier King']

In [14]:
locations, names, levels, scores, review_counts, room_types, room_details, prices_1, loading_urls = [], [], [], [], [] ,[], [], [], []

num1 = 0
for link in links[:10]:
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        req = requests.get(link, headers=headers)
        response = TextResponse(req.url, body=req.text, encoding="utf-8")

        location = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        location = json.loads(location)["props"]["pageProps"]["placeInfo"]["address"]
        name = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        level = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        score = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        review_count = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_type = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract() == []:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:
            room_detail = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[2]/text()').extract()
        try:
            price = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[4]/div/div[5]/span/text()').extract()
            prices=[]
            for num in range(len(room_type)):
                price = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
                price = json.loads(price)["props"]["pageProps"]["placeInfo"]["rooms"][num]["prices"][0]["price"]["discount"]["price"]
                prices.append(price)
        except:
            prices= ["예약마감"]
    except:
        location = "페이지 없음"
        location = "페이지 없음"
        name = "페이지 없음"
        level = "페이지 없음"
        score = "페이지 없음"
        review_count = "페이지 없음"
        room_type = ["페이지 없음"]
        room_detail = ["페이지 없음"]
        price = ["페이지 없음"]
        prices= ["페이지 없음"]
        
    location = [location] * len(room_type)
    name = [name] * len(room_type)
    level = [level] * len(room_type)
    score = [score] * len(room_type)
    review_count = [review_count] * len(room_type)
    link = [link] * len(room_type)
    
    locations.extend(location)
    names.extend(name)
    levels.extend(level)
    scores.extend(score)
    review_counts.extend(review_count)
    room_types.extend(room_type)
    room_details.extend(room_detail)
    prices_1.extend(prices)
    loading_urls.extend(link)
    num1+=1
    print(str(num1) ,name[0])

1 페어필드 바이 메리어트 서울
2 나인트리 프리미어 명동2
3 신라스테이 구로
4 영등포 GMS 호텔
5 글래드 마포
6 노보텔 앰배서더 서울 용산
7 신라스테이 마포
8 홀리데이인 익스프레스 서울 홍대
9 더 파크호텔 서울
10 호텔 더 디자이너스 여의도


In [76]:
df = pd.DataFrame({
    'Name': names, 'Level': levels, 'Score':scores, 'Location': locations,
    'RoomType': room_types,'Roomdetail': room_details ,'Price': prices_1, 'Link': loading_urls, 'Review' : review_counts,
})
df.tail()

,Name,Level,Score,Location,RoomType,Roomdetail,Price,Link,Review
3388,종로 호텔 팝,비즈니스,5.0,서울특별시 종로구 수표로 22길 16,디럭스 더블,주차가능/넷플릭스,"62,360원",https://www.yanolja.com/hotel/1000111306,1
3389,종로 호텔 팝,비즈니스,5.0,서울특별시 종로구 수표로 22길 16,프리미어 테라스 더블,주차가능/넷플릭스,"74,120원",https://www.yanolja.com/hotel/1000111306,1
3390,종로 호텔 팝,비즈니스,5.0,서울특별시 종로구 수표로 22길 16,스위트 테라스 더블,주차가능/넷플릭스,"85,890원",https://www.yanolja.com/hotel/1000111306,1
3391,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,https://www.yanolja.com/hotel/1000109348,페이지 없음
3392,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,페이지 없음,https://www.yanolja.com/hotel/1000112890,페이지 없음


### --------확인 끝 spider.py로 확장---------

In [19]:
# 3. items.py : title, content, link, category

In [109]:
%%writefile yanolja_hotel/yanolja_hotel/items.py
import scrapy

class YanoljaHotelItem(scrapy.Item):
    location = scrapy.Field()
    name = scrapy.Field()
    level = scrapy.Field()
    score = scrapy.Field()
    review_count = scrapy.Field()
    link = scrapy.Field()
    room_type = scrapy.Field()
    flatform = scrapy.Field()

Overwriting yanolja_hotel/yanolja_hotel/items.py


In [110]:
# 4. spider.py

In [73]:
%%writefile yanolja_hotel/yanolja_hotel/spiders/spider.py
import scrapy
from yanolja_hotel.items import YanoljaHotelItem
from yanolja import get_hotel_keys
import json
import datetime

class Spider(scrapy.Spider):
    name = "YanoljaHotel"
    allow_domain = ["https://www.yanolja.com"]
    start_urls = ["https://www.yanolja.com/hotel/"]
    
    def parse(self, response):
        hotel_keys = get_hotel_keys()
        start_urls = "https://www.yanolja.com/hotel/"
        links = []
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"}
        for hotel_key in hotel_keys:
            hotel_url = start_urls + hotel_key
            links.append(hotel_url)
        for link in links:
            yield scrapy.Request(link, headers=headers, callback=self.parse_content)
    
    def parse_content(self, response):
        if response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract() == []:
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/text()[1]').extract()
        else:    
            room_types = response.xpath('//*[@id="__next"]/main/div[1]/div[1]/section/div/div[2]/div[1]/h2/text()').extract()
        locations = response.xpath('//*[@id="__NEXT_DATA__"]/text()')[0].extract()
        locations = json.loads(locations)["props"]["pageProps"]["placeInfo"]["address"]
        locations = [locations] * len(room_types)
        names = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/h1/text()')[0].extract()
        names = [names] * len(room_types)
        levels = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/div[1]/div[1]/text()')[0].extract()
        levels = [levels] * len(room_types)
        scores = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/div/span[1]/text()')[0].extract()
        scores = [scores] * len(room_types)
        review_counts = response.xpath('//*[@id="__next"]/main/div[1]/section[2]/a/span/b/text()')[0].extract()
        review_counts = [review_counts] * len(room_types)
        links = response.url
        links = [links] * len(room_types)
        flatforms = response.url.split(".")[1]
        flatforms = [flatforms] * len(room_types)
        for flatform,name,level,location,score,review_count,link,room_type in zip(flatforms,names,levels,locations,scores,review_counts,links,room_types):
            yield {"flatform":flatform,"name":name, "level":level, "location":location, "score":score, "review_count":review_count,"roomtype":room_type, "link":link}

Overwriting yanolja_hotel/yanolja_hotel/spiders/spider.py


In [74]:
# 5. settings.py : robots.txt 여부 파악 

In [75]:
# 6. 프로젝트 실행

In [76]:
%%writefile run.sh
cd yanolja_hotel
rm yanolja_hotel.csv
scrapy crawl YanoljaHotel -o yanolja_hotel.csv

Overwriting run.sh


In [77]:
!/bin/bash run.sh

2021-03-10 00:10:10 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: yanolja_hotel)
2021-03-10 00:10:10 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.6.9 (default, Jan 15 2021, 05:57:06) - [GCC 7.5.0], pyOpenSSL 20.0.1 (OpenSSL 1.1.1i  8 Dec 2020), cryptography 3.3.1, Platform Linux-5.4.0-1038-aws-x86_64-with-debian-buster-sid
2021-03-10 00:10:10 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2021-03-10 00:10:10 [scrapy.crawler] INFO: Overridden settings:
{'BOT_NAME': 'yanolja_hotel',
 'NEWSPIDER_MODULE': 'yanolja_hotel.spiders',
 'SPIDER_MODULES': ['yanolja_hotel.spiders']}
2021-03-10 00:10:10 [scrapy.extensions.telnet] INFO: Telnet Password: 6c0745f81679d551
2021-03-10 00:10:10 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy

2021-03-10 00:10:13 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-10&checkoutDate=2021-03-11&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=12&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-10 00:10:13 [urllib3.connectionpool] DEBUG: Starting new HTTPS connection (1): www.yanolja.com:443
2021-03-10 00:10:13 [urllib3.connectionpool] DEBUG: https://www.yanolja.com:443 "GET /api/v1/contents/search?advert=KEYWORD&capacityAdult=2&capacityChild=0%20%20%20%20&checkinDate=2021-03-10&checkoutDate=2021-03-11&hotel=1&keyword=%EC%84%9C%EC%9A%B8&lat=37.50681&lng=127.06624&page=13&limit=20&%20%20%20%20searchKeyword=%EC%84%9C%EC%9A%B8&searchType=hotel&sort=100&pathDivision=keyword-hotel&themes= HTTP/1.1" 200 None
2021-03-10 00:10:13 [urllib3.connectionpool] DEBUG: Sta

2021-03-10 00:10:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012015>
{'flatform': 'yanolja', 'name': '글래드 강남 코엑스센터', 'level': '3성급', 'location': '서울특별시 강남구 테헤란로 610', 'score': '4.7', 'review_count': '1,540', 'roomtype': '스탠다드 트윈', 'link': 'https://www.yanolja.com/hotel/3012015'}
2021-03-10 00:10:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012015>
{'flatform': 'yanolja', 'name': '글래드 강남 코엑스센터', 'level': '3성급', 'location': '서울특별시 강남구 테헤란로 610', 'score': '4.7', 'review_count': '1,540', 'roomtype': '스탠다드 더블', 'link': 'https://www.yanolja.com/hotel/3012015'}
2021-03-10 00:10:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012015>
{'flatform': 'yanolja', 'name': '글래드 강남 코엑스센터', 'level': '3성급', 'location': '서울특별시 강남구 테헤란로 610', 'score': '4.7', 'review_count': '1,540', 'roomtype': '슈페리어 더블', 'link': 'https://www.yanolja.com/hotel/3012015'}
2021-03-10 00:10:15 [scrapy.core.scraper] DEBUG: 

2021-03-10 00:10:15 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3000643>
{'flatform': 'yanolja', 'name': '호텔 피제이 명동', 'level': '4성급', 'location': '서울특별시 중구 마른내로 71', 'score': '4.7', 'review_count': '1,674', 'roomtype': '스탠다드 트윈', 'link': 'https://www.yanolja.com/hotel/3000643'}
2021-03-10 00:10:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3013410>
{'flatform': 'yanolja', 'name': '페어필드 바이 메리어트 서울', 'level': '4성급', 'location': '서울특별시 영등포구 경인로 870', 'score': '4.7', 'review_count': '8,785', 'roomtype': 'Premier King', 'link': 'https://www.yanolja.com/hotel/3013410'}
2021-03-10 00:10:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3013410>
{'flatform': 'yanolja', 'name': '페어필드 바이 메리어트 서울', 'level': '4성급', 'location': '서울특별시 영등포구 경인로 870', 'score': '4.7', 'review_count': '8,785', 'roomtype': 'Premier King', 'link': 'https://www.yanolja.com/hotel/3013410'}
2021-03-10 00:10:16 [scrapy.core.scrap

2021-03-10 00:10:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001397>
{'flatform': 'yanolja', 'name': '신라스테이 동탄', 'level': '3성급', 'location': '경기도 화성시 노작로 161', 'score': '4.7', 'review_count': '1,619', 'roomtype': '17-22시 당일이용/5시간 STAY-스탠다드 체크인시 배정', 'link': 'https://www.yanolja.com/hotel/3001397'}
2021-03-10 00:10:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001397>
{'flatform': 'yanolja', 'name': '신라스테이 동탄', 'level': '3성급', 'location': '경기도 화성시 노작로 161', 'score': '4.7', 'review_count': '1,619', 'roomtype': '17-22시 당일이용/5시간 STAY-스탠다드 체크인시 배정+코바커피 2잔', 'link': 'https://www.yanolja.com/hotel/3001397'}
2021-03-10 00:10:16 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001397>
{'flatform': 'yanolja', 'name': '신라스테이 동탄', 'level': '3성급', 'location': '경기도 화성시 노작로 161', 'score': '4.7', 'review_count': '1,619', 'roomtype': '스탠다드 트윈', 'link': 'https://www.yanolja.com/hotel/3001397'}
2021-03-10

...

2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,477', 'roomtype': '수페리어 트윈', 'link': 'https://www.yanolja.com/hotel/3016365'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,477', 'roomtype': '수페리어 퀸 발코니', 'link': 'https://www.yanolja.com/hotel/3016365'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3016365>
{'flatform': 'yanolja', 'name': '더 파크호텔 서울', 'level': '4성급', 'location': '서울특별시 영등포구 양평로 136', 'score': '4.6', 'review_count': '1,477', 'roomtype': '수페리어 수퍼 킹', 'link': 'https://www.yanolja.com/hotel/3016365'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scra

2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja', 'name': '신라스테이 마포', 'level': '3성급', 'location': '서울특별시 마포구 마포대로 83', 'score': '4.6', 'review_count': '3,157', 'roomtype': '스탠다드→디럭스 더블', 'link': 'https://www.yanolja.com/hotel/3001415'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja', 'name': '신라스테이 마포', 'level': '3성급', 'location': '서울특별시 마포구 마포대로 83', 'score': '4.6', 'review_count': '3,157', 'roomtype': '체크인시 객실 배정', 'link': 'https://www.yanolja.com/hotel/3001415'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001415>
{'flatform': 'yanolja', 'name': '신라스테이 마포', 'level': '3성급', 'location': '서울특별시 마포구 마포대로 83', 'score': '4.6', 'review_count': '3,157', 'roomtype': '스탠다드 트윈', 'link': 'https://www.yanolja.com/hotel/3001415'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped 

2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012800>
{'flatform': 'yanolja', 'name': '글래드 마포', 'level': '4성급', 'location': '서울특별시 마포구 마포대로 92', 'score': '4.8', 'review_count': '2,921', 'roomtype': '내맘대로 9시간 STAY 점보 트윈', 'link': 'https://www.yanolja.com/hotel/3012800'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012800>
{'flatform': 'yanolja', 'name': '글래드 마포', 'level': '4성급', 'location': '서울특별시 마포구 마포대로 92', 'score': '4.8', 'review_count': '2,921', 'roomtype': '내맘대로 9시간 STAY 디럭스 트윈', 'link': 'https://www.yanolja.com/hotel/3012800'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3012800>
{'flatform': 'yanolja', 'name': '글래드 마포', 'level': '4성급', 'location': '서울특별시 마포구 마포대로 92', 'score': '4.8', 'review_count': '2,921', 'roomtype': '내맘대로 9시간 STAY 스탠다드 더블', 'link': 'https://www.yanolja.com/hotel/3012800'}
2021-03-10 00:10:36 [scrapy.cor

2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja', 'name': '신라스테이 구로', 'level': '3성급', 'location': '서울특별시 동작구 시흥대로 596', 'score': '4.6', 'review_count': '4,108', 'roomtype': '스탠다드 더블', 'link': 'https://www.yanolja.com/hotel/3001564'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja', 'name': '신라스테이 구로', 'level': '3성급', 'location': '서울특별시 동작구 시흥대로 596', 'score': '4.6', 'review_count': '4,108', 'roomtype': '스탠다드 더블', 'link': 'https://www.yanolja.com/hotel/3001564'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3001564>
{'flatform': 'yanolja', 'name': '신라스테이 구로', 'level': '3성급', 'location': '서울특별시 동작구 시흥대로 596', 'score': '4.6', 'review_count': '4,108', 'roomtype': '프리미어 스탠다드 더블', 'link': 'https://www.yanolja.com/hotel/3001564'}
2021-03-10 00:10:36 [scrapy.core.scraper] DEBUG: Scraped

2021-03-10 00:10:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja', 'name': '나인트리 프리미어 명동2', 'level': '4성급', 'location': '서울특별시 중구 마른내로 28', 'score': '4.8', 'review_count': '2,185', 'roomtype': '스탠다드 트윈', 'link': 'https://www.yanolja.com/hotel/3009497'}
2021-03-10 00:10:37 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.yanolja.com/hotel/3009497>
{'flatform': 'yanolja', 'name': '나인트리 프리미어 명동2', 'level': '4성급', 'location': '서울특별시 중구 마른내로 28', 'score': '4.8', 'review_count': '2,185', 'roomtype': '스탠다드 더블', 'link': 'https://www.yanolja.com/hotel/3009497'}
2021-03-10 00:10:37 [scrapy.core.engine] INFO: Closing spider (finished)
2021-03-10 00:10:37 [scrapy.extensions.feedexport] INFO: Stored csv feed (3428 items) in: yanolja_hotel.csv
2021-03-10 00:10:37 [scrapy.statscollectors] INFO: Dumping Scrapy stats:
{'downloader/request_bytes': 214051,
 'downloader/request_count': 417,
 'downloader/request_method_count/GET': 417

In [71]:
data = pd.read_csv('./yanolja_hotel/yanolja_hotel.csv')

In [72]:
data

,flatform,name,level,location,score,review_count,roomtype,link
0,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 트윈 + 레이트체크아웃,https://www.yanolja.com/hotel/3009495
1,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 더블 + 레이트체크아웃,https://www.yanolja.com/hotel/3009495
2,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 트윈,https://www.yanolja.com/hotel/3009495
3,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 더블,https://www.yanolja.com/hotel/3009495
4,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",디럭스 더블 + 레이트체크아웃,https://www.yanolja.com/hotel/3009495
5,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 트윈,https://www.yanolja.com/hotel/3009495
6,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 더블,https://www.yanolja.com/hotel/3009495
7,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",디럭스 더블,https://www.yanolja.com/hotel/3009495
8,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 트윈,https://www.yanolja.com/hotel/3009495
9,yanolja,홍대 나인브릭 호텔,3성급,서울특별시 마포구 홍익로5길 32,4.7,"1,463",슈페리어 더블,https://www.yanolja.com/hotel/3009495


In [ ]:
# 7. pipeline.py : 크롤링한 데이터를 mysql에 저장

# -----------TEST------------